In [54]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [55]:
block_size=8
batch_size=4
max_iter=10000
learning_rate=3e-4
n_embd=384
n_layer=4

In [56]:
with open('wizard_of_oz.txt','r',encoding='utf-8') as f:
    text=f.read()

In [57]:
char=sorted(set(text))
print(' the length of char is {}'.format(len(char)))
print(char)
vocab_size=len(char)

 the length of char is 80
['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [58]:
string_to_int={ch:i for i, ch in enumerate(char)}
int_to_string={i:ch for i, ch in enumerate(char)}
encode=lambda s:[string_to_int[c] for c in s]
decode=lambda s:"".join([int_to_string[c] for c in s])

In [59]:
data = torch.tensor(encode(text), dtype=torch.long)

In [60]:
n=int(len(text)*0.8)
train_data=data[:n]
val_data=data[n:]

In [61]:
#Get batch function
def get_batch(split):
    data=train_data if split=='train' else val_data
    ix=torch.randint(len(data)-block_size,(batch_size,))
    #print(ix)
    x=torch.stack([data[i:i+block_size] for i in ix])
    y=torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y


In [66]:
class GptLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table=nn.Embedding(vocab_size,n_embd)
        self.position_embedding_table=nn.Embedding(block_size,n_embd)
        self.block=nn.Sequential(*[block(n_embd,n_head=n_head) for _ in range(n_layer)])
        self.ln_f=nn.LayerNorm(n_embd) #final layer norm
        self.lm_head=nn.Linear(n_embd,vocab_size)
    
    def forward(self,index,target=None):
       
       token_emb=self.token_embedding_table(idx)
       pos_emb=self.position_embedding_table(torch.arrange(T)) 
       x=token_emb+pos_emb
       x=self.block(x)
       x=self.ln_f(x)
       logit=self.lm_head(x)

       if target==None:
            loss=None
       else:
         #geting shape of logits
         B, T, C=logit.shape
         logit=logit.view(B*T,C)
         target=target.view(B*T)
         loss=F.cross_entropy(logit,target)
        
       return logit, loss
    
    def generate(self,index,max_new_tokens):
        #index is (B,T) array of indices in the current content
        for _ in range(max_new_tokens):
            #get prediction
            logit, loss=self.forward(index)
            #focus only on last step
            logit=logit[:,-1,:] #becomes (B,C)
            probs=F.softmax(logit,dim=-1) #(B,C)
            index_next=torch.multinomial(probs,num_samples=1)
            #append sampled index to the running sequence
            index=torch.cat((index,index_next),dim=1)
        return index


In [63]:
model=GptLanguageModel(vocab_size)

NameError: name 'block' is not defined

In [72]:
print(torch.empty(1,1,1))

tensor([[[1.0286e-38]]])


In [ ]:
context=torch.zeros((1,1),dtype=torch.long,)

In [ ]:
generated_char=decode(model.generate(
    context,
    max_new_tokens=500)[0].tolist()
)
print(generated_char)



ro yowowa so o t idokld bl

oreinthy thy't
" a f ew,"y-helevex.

"
arerkemun s.
ane f ad y bend mer t, s ve om."Ye bufry.
ABunge t omed tt wontllimbeey,  ch.

"E STe wouseacerdouse.

des. aut ve the
Zou thed fis theed borr thacewenelllosene om, g'ld ntharwo ld--om, wlavo wad aler aine thats it e e abr Ther stonthand ato
Th thave thy ME
exche bifil a t mad

f ded Ther th scthealcthangouted thamanoougas ad aply,"Ozmpizas. y.
AX1. re th ot h o," y thee fled tond. tht  towanhan

Th rt, 1786.
cey; we


In [ ]:

optimizer=torch.optim.AdamW(model.parameters(),lr=learning_rate)
losst=[]

for iter in range(max_iter*2):
    xb, yb=get_batch('train')

    #Evluate loss

    logit, loss=model.forward(xb,yb)
    losst.append(loss)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss)

tensor(2.7099, grad_fn=<NllLossBackward0>)
